In [ ]:
import numpy as np
import math

In [9]:
# randomly generate x,y
x = np.random.rand(2)
y = (x[0]**2 + x[1]**2)/2

# first hidden layer
w01 = np.random.rand(10,2)
b01 = np.random.rand(10,1)

# second hidden layer
w12 = np.random.rand(10)
b12 = np.random.rand(10)
print(w01)
print(b01)
print(w12)
print(b12)

[[0.11053824 0.43232863]
 [0.99456011 0.12556181]
 [0.3831096  0.1318824 ]
 [0.92371145 0.57388239]
 [0.60093944 0.60921293]
 [0.87580652 0.45987647]
 [0.91286448 0.33270179]
 [0.23317415 0.64302704]
 [0.86656281 0.88017308]
 [0.00612996 0.91618516]]
[[0.15248329]
 [0.98792046]
 [0.59144739]
 [0.37255404]
 [0.72077377]
 [0.2349931 ]
 [0.95383343]
 [0.7828949 ]
 [0.29868455]
 [0.97469718]]
[0.83822323 0.38237847 0.08309667 0.27926363 0.36317496 0.71240211
 0.60388677 0.97586789 0.54392889 0.67744885]
[0.95259594 0.78566764 0.71508619 0.35556093 0.40253866 0.46308107
 0.04647624 0.02631424 0.54417368 0.51052751]


In [28]:
def sigmoid(x):
  # sigmoid function
  return 1 / (1 + math. exp(-x))

def forward01(x,w,b,act):
  # first forward pass 
  z = x * w + b
  v = []
  for i in range(len(z)):
    vv = act(z[i][0] + z[i][1])
    v.append(vv)
  return v,z 

def forward12(x,w,b,act):
  # second forward pass
  z = x * w + b
  v = []
  for i in range(len(z)):
    vv = act(z[i])
    v.append(vv)
  return np.sum(v),z 

def loss(pred,y):
  # loss function
  L = (pred- y)**2
  return L

In [59]:
# dL/dw12 = 2(yred-y) * sig(z2) * (1-sig(z2)) * -v1
def gradient_w12(v,y,pred,z2):
  dw12 = []
  sigz2 = []
  for i in range(10):
    sigz = sigmoid(z2[i]) * (1 - sigmoid(z2[i]))
    sigz2.append(sigz)
    dw = -2 *v[i]* (pred - y) * sigz
    dw12.append(dw)
  return np.array(dw12),np.array(sigz2)
# dL/db12 = 2 * (yred - y)*sig(z1)(1-sig(z1))
def gradient_b12(y,pred,sigz2):
  db12 = []
  for i in range(10):
    db = 2 * (pred - y)* sigz2[i]
    db12.append(db)
  return np.array(db12)

In [74]:
# dL/dw01 = 2 * (yred - y)* sig(z2) * w12 * sig(z1) * -x
def gradient_w01(x,y,w12,z1,pred,sigz2):
  dw01 = []
  sigz1 = []
  for i in range(10):
    sigz = sigmoid(z1[i][0] + z1[i][1]) * (1 - sigmoid(z1[i][0] + z1[i][1]))
    sigz1.append(sigz)
    dw011 = -2 * x[0] * (pred - y) * sigz2[i] * w12[i] * sigz
    dw012 = -2 * x[1] * (pred - y) * sigz2[i] * w12[i] * sigz
    dw01.append([dw011,dw012])
  return np.array(dw01),np.array(sigz1)

# dL/db01 = 2 * (yred - y)*sig(z2)* w12 * sig(z1)
def gradient_b01(y,pred,sigz2,sigz1,w12):
  db01 = []
  for i in range(10):
    db = 2 * (pred - y)* sigz2[i] * w12[i] * sigz1[i]
    db01.append([db])
  return np.array(db01)

In [75]:
# forward 
v1,z1 = forward01(x,w01,b01,sigmoid)
y_pred,z2 = forward12(v1,w12,b12,sigmoid)

# loss
L = loss(y_pred,y)

# gradient decent
dw12,sigz2 = gradient_w12(v1,y,y_pred,z2)
db12 = gradient_b12(y,y_pred,sigz2)
dw01, sigz1 = gradient_w01(x,y,w12,z1,y_pred,sigz2)
db01 = gradient_b01(y,y_pred,sigz2,sigz1,w12)

print(dw12)
print(db12)
print(dw01)
print(db01)



[-1.31585083 -2.30514521 -2.37490431 -2.62956265 -2.66130537 -2.08071665
 -2.86930525 -2.46325871 -2.24912784 -2.27994283]
[1.98641011 2.44740055 2.87881518 3.06363502 2.93195317 2.58579292
 3.04458197 2.74607448 2.6150319  2.45550307]
[[-0.27255531 -0.24861507]
 [-0.03783732 -0.03451383]
 [-0.026648   -0.02430734]
 [-0.07735436 -0.07055984]
 [-0.06605971 -0.06025727]
 [-0.21251504 -0.19384851]
 [-0.07348467 -0.06703005]
 [-0.18156788 -0.16561964]
 [-0.12594543 -0.11488285]
 [-0.08112589 -0.0740001 ]]
[[0.37350358]
 [0.0518514 ]
 [0.03651781]
 [0.10600465]
 [0.09052672]
 [0.29122576]
 [0.10070171]
 [0.24881648]
 [0.17259274]
 [0.11117307]]


In [68]:
# calculate gradient decent with torch
import torch
x_train = torch.tensor(x)
y_train = torch.tensor(y)

tw01 = torch.tensor(w01, requires_grad=True)
tb01 = torch.tensor(b01, requires_grad=True)
tw12 = torch.tensor(w12, requires_grad=True)
tb12 = torch.tensor(b12, requires_grad=True)

m = torch.nn.Sigmoid()

# forward 
v1,z = forward01(x_train,tw01,tb01,m)
# the result of v1 is a list, need to convert it to tensor
ve = np.array(v1)
tv1 = torch.tensor(v1)
tpred,z = forward12(tv1,tw12,tb12,m)

# loss
L = loss(tpred,y_train)

# backward
L.backward()

print(tpred)
print(tw01.grad)
print(tb01.grad)
print(tw12.grad)
print(tb12.grad)


tensor(7.1734, dtype=torch.float64, grad_fn=<AddBackward0>)
None
None
tensor([1.3143, 2.3053, 2.3741, 2.6292, 2.6613, 2.0795, 2.8698, 2.4630, 2.2485,
        2.2800], dtype=torch.float64)
tensor([1.9845, 2.4476, 2.8779, 3.0634, 2.9320, 2.5846, 3.0451, 2.7459, 2.6144,
        2.4556], dtype=torch.float64)


In [83]:
n_iters = 20
learning_rate = 0.1

In [84]:

for epoch in range(n_iters):
  # prediction = forward pass
  v1,z1 = forward01(x,w01,b01,sigmoid)
  y_pred,z2 = forward12(v1,w12,b12,sigmoid)

  # loss
  L = loss(y_pred,y)

  # gradients:
  dw12,sigz2 = gradient_w12(v1,y,y_pred,z2)
  db12 = gradient_b12(y,y_pred,sigz2)
  dw01, sigz1 = gradient_w01(x,y,w12,z1,y_pred,sigz2)
  db01 = gradient_b01(y,y_pred,sigz2,sigz1,w12)

  # update w and b 
  w12 -= learning_rate * dw12
  b12 -= learning_rate * db12
  w01 -= learning_rate * dw01
  b01 -= learning_rate * db01

  if epoch % 1 == 0:
    print(f'epoch{epoch + 1}: loss = {L}')


epoch1: loss = 17.707679035956602
epoch2: loss = 15.3698496737384
epoch3: loss = 13.307216919282205
epoch4: loss = 11.517127244131428
epoch5: loss = 9.983053069005745
epoch6: loss = 8.680782228825905
epoch7: loss = 7.581442466758169
epoch8: loss = 6.654340639607789
epoch9: loss = 5.869925732895914
epoch10: loss = 5.201986919329141
epoch11: loss = 4.628628633946717
epoch12: loss = 4.132254735269113
epoch13: loss = 3.699035169035062
epoch14: loss = 3.318221576244457
epoch15: loss = 2.9814963448116294
epoch16: loss = 2.682415237626347
epoch17: loss = 2.4159479759895905
epoch18: loss = 2.178106069903481
epoch19: loss = 1.9656478002895905
epoch20: loss = 1.7758531921408391
